 # Exercise 5.4.8 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize ,poly)
import statsmodels.api as sm

We will now perform cross-validation on a simulated data set.

a. Generate a simulated data set with the given code. In this data set, what is n and what is p? Write out the model used to generate the data in equation form.

In [ ]:
#8a
rng = np.random.default_rng(1)
x = rng.normal(size=100)
y = x - 2 * x**2 + rng.normal(size=100)

$ y = 0 + x - 2x^2 + \epsilon $

b. Create a scatterplot of X against Y.

In [ ]:
#8b
plt.scatter(x, y)
plt.title('X vs Y')
plt.show()
#clearly non-linear, quadratic function.

c. Compute the LOOCV errors that result from fitting the following four models using least squares. (d) Repeat (c) using another random seed, and report your results. Are your results the same as what you got in (c)? Why? (e) Which of the models in (c) had the smallest LOOCV error? Is this what you expected? Explain your answer. Comment on the statistical significance of the coefficient estimates that results from fitting each of the models in (c) using least squares.

In [ ]:
#8c &d
rng = np.random.default_rng(10)
x = rng.normal(size=100)
y = x - 2 * x**2 + rng.normal(size=100)
df= pd.DataFrame({'Y':y,'X':x,'X2':x**2,'X3':x**3,'X4':x**4})
M_CV1 = cross_validate(LinearRegression(), df[['X']], y, cv=len(x), scoring='neg_mean_squared_error')
M_CV2 = cross_validate(LinearRegression(), df[['X','X2']], y, cv=len(x), scoring='neg_mean_squared_error')
M_CV3 = cross_validate(LinearRegression(), df[['X','X2','X3']], y, cv=len(x), scoring='neg_mean_squared_error')
M_CV4 = cross_validate(LinearRegression(), df[['X','X2','X3','X4']], y, cv=len(x), scoring='neg_mean_squared_error')
y=[-np.mean(M_CV1['test_score']),-np.mean(M_CV2['test_score']),-np.mean(M_CV3['test_score']),-np.mean(M_CV4['test_score'])]
print(y)
plt.plot(["X","X2","X3","X4"], y)
plt.show()

As you change the seed the dataset changes slightly due to the calls to rnorm. The LOOCV has a deterministic definition of the folds so this has no influence on the changes. In the example case (on my system) the minimal score is for a quadratic model, which is correct, and the statistical Wald tests for the coefficients also point to evidence for only a quadratic model. Changing the seed can cause the minimal model going by minimal LOOCV MSE score to sometimes change, mostly because the differences between the differently sized models (2 and larger) are very small and within the variability of MSE estimates across incarnations of the data. The MSE estimates are Random Variables, so one should be aware of the variability in these estimates.

In [ ]:
model = smf.glm("Y ~ X + X2 +X3 + X4", data=df).fit()
print(model.summary())


For some version of the data X4 can also be significant. One should keep in mind that X2 and X4 are correlated variables and can both explain the same trend over the data range.